<a href="https://colab.research.google.com/github/AnhQuocVo/China-Real-Estate-Prediction/blob/main/Real_Estate_Demand_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

china_real_estate_demand_prediction_path = kagglehub.competition_download('china-real-estate-demand-prediction')

print('Data source import complete.')


## 1. IMPORTS & CONFIGURATION

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/china-real-estate-demand-prediction/sample_submission.csv
/kaggle/input/china-real-estate-demand-prediction/test.csv
/kaggle/input/china-real-estate-demand-prediction/train/city_search_index.csv
/kaggle/input/china-real-estate-demand-prediction/train/land_transactions_nearby_sectors.csv
/kaggle/input/china-real-estate-demand-prediction/train/new_house_transactions_nearby_sectors.csv
/kaggle/input/china-real-estate-demand-prediction/train/city_indexes.csv
/kaggle/input/china-real-estate-demand-prediction/train/pre_owned_house_transactions.csv
/kaggle/input/china-real-estate-demand-prediction/train/new_house_transactions.csv
/kaggle/input/china-real-estate-demand-prediction/train/land_transactions.csv
/kaggle/input/china-real-estate-demand-prediction/train/sector_POI.csv
/kaggle/input/china-real-estate-demand-prediction/train/pre_owned_house_transactions_nearby_sectors.csv


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import gc

# ML Libraries
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor

# Hyperparameter Optimization
try:
    import optuna
    OPTUNA_AVAILABLE = True
except ImportError:
    OPTUNA_AVAILABLE = False
    print("Optuna not available. Using default hyperparameters.")

In [ ]:
# Configuration
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")


## 2. DATA LOADING & OVERVIEW

In [ ]:
def load_data():
    # Training data
    new_house = pd.read_csv('/kaggle/input/china-real-estate-demand-prediction/train/new_house_transactions.csv')
    new_house_nearby = pd.read_csv('/kaggle/input/china-real-estate-demand-prediction/train/new_house_transactions_nearby_sectors.csv')
    preowned_house = pd.read_csv('/kaggle/input/china-real-estate-demand-prediction/train/pre_owned_house_transactions.csv')
    preowned_nearby = pd.read_csv('/kaggle/input/china-real-estate-demand-prediction/train/pre_owned_house_transactions_nearby_sectors.csv')
    land_trans = pd.read_csv('/kaggle/input/china-real-estate-demand-prediction/train/land_transactions.csv')
    land_nearby = pd.read_csv('/kaggle/input/china-real-estate-demand-prediction/train/land_transactions_nearby_sectors.csv')
    sector_poi = pd.read_csv('/kaggle/input/china-real-estate-demand-prediction/train/sector_POI.csv')
    city_search = pd.read_csv('/kaggle/input/china-real-estate-demand-prediction/train/city_search_index.csv')
    city_indexes = pd.read_csv('/kaggle/input/china-real-estate-demand-prediction/train/city_indexes.csv')

    # Test data
    test = pd.read_csv('/kaggle/input/china-real-estate-demand-prediction/test.csv')
    sample_submission = pd.read_csv('/kaggle/input/china-real-estate-demand-prediction/sample_submission.csv')

    print("✓ All datasets loaded successfully!")

    datasets = {
        'new_house': new_house,
        'new_house_nearby': new_house_nearby,
        'preowned_house': preowned_house,
        'preowned_nearby': preowned_nearby,
        'land_trans': land_trans,
        'land_nearby': land_nearby,
        'sector_poi': sector_poi,
        'city_search': city_search,
        'city_indexes': city_indexes,
        'test': test,
        'sample_submission': sample_submission
    }

    return datasets

def data_overview(datasets):
    for name, df in datasets.items():
        if name != 'sample_submission':
            print(f"\n{name.upper()}:")
            print(f"  Shape: {df.shape}")
            print(f"  Columns: {list(df.columns)}")
            print(f"  Missing values: {df.isnull().sum().sum()}")
            print(f"  Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Load data
data = load_data()
data_overview(data)

## 3. EXPLORATORY DATA ANALYSIS (EDA)

In [ ]:
def parse_month_sector(df):
    df = df.copy()
    df['month'] = df['month_sector'].str.extract(r'(\d{4}\s+\w+)')[0]
    df['sector'] = df['month_sector'].str.extract(r'sector\s+(\d+)')[0].astype(int)
    df['month'] = pd.to_datetime(df['month'], format='%Y %b')
    return df

In [ ]:
def eda_analysis(data):
    # Parse target data
    new_house = parse_month_sector(['new_house'])

    print("\n1. TARGET VARIABLE ANALYSIS")
    print("-" * 80)
    target = 'amount_new_house_transaction'
    print(f"\nTarget: {target}")
    print(new_house[target].describe())

    # Time series plot
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))

    # Overall trend
    monthly_avg = new_house.groupby('month')[target].mean()
    axes[0, 0].plot(monthly_avg.index, monthly_avg.values, marker='o', linewidth=2)
    axes[0, 0].set_title('Average New House Transactions Over Time', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Month')
    axes[0, 0].set_ylabel('Average Transaction Amount')
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].tick_params(axis='x', rotation=45)

    # Distribution
    axes[0, 1].hist(new_house[target], bins=50, edgecolor='black', alpha=0.7)
    axes[0, 1].set_title('Distribution of Transaction Amounts', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Transaction Amount')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].grid(True, alpha=0.3)

    # Top sectors
    sector_avg = new_house.groupby('sector')[target].mean().sort_values(ascending=False).head(10)
    axes[1, 0].barh(sector_avg.index.astype(str), sector_avg.values, color='teal')
    axes[1, 0].set_title('Top 10 Sectors by Average Transaction', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Average Transaction Amount')
    axes[1, 0].set_ylabel('Sector')
    axes[1, 0].grid(True, alpha=0.3, axis='x')

    # Box plot by year
    new_house['year'] = new_house['month'].dt.year
    axes[1, 1].boxplot([new_house[new_house['year'] == y][target].dropna()
                        for y in sorted(new_house['year'].unique())],
                       labels=sorted(new_house['year'].unique()))
    axes[1, 1].set_title('Transaction Amount Distribution by Year', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Year')
    axes[1, 1].set_ylabel('Transaction Amount')
    axes[1, 1].grid(True, alpha=0.3, axis='y')

    plt.tight_layout()
    plt.savefig('eda_overview.png', dpi=300, bbox_inches='tight')
    print("\n✓ EDA plots saved as 'eda_overview.png'")

    print("\n2. KEY INSIGHTS")
    print("-" * 80)
    print(f"  • Total unique sectors: {new_house['sector'].nunique()}")
    print(f"  • Date range: {new_house['month'].min()} to {new_house['month'].max()}")
    print(f"  • Average transaction amount: {new_house[target].mean():.2f}")
    print(f"  • Transaction amount std: {new_house[target].std():.2f}")
    print(f"  • Coefficient of Variation: {(new_house[target].std() / new_house[target].mean()):.2%}")

# Run EDA
eda_analysis(data)

NameError: name 'data' is not defined

## 4. FEATURE ENGINEERING

In [ ]:

def engineer_features(data):
    """Comprehensive feature engineering pipeline"""

    print("\nEngineering features...")

    # Parse all relevant datasets
    new_house = parse_month_sector(data['new_house'])
    preowned = parse_month_sector(data['preowned_house'])
    land = parse_month_sector(data['land_trans'])
    test_df = parse_month_sector(data['test'])

    # Combine train and test for consistent feature engineering
    new_house['is_train'] = 1
    test_df['is_train'] = 0
    test_df['amount_new_house_transaction'] = np.nan

    combined = pd.concat([new_house, test_df], axis=0, ignore_index=True)
    combined = combined.sort_values(['sector', 'month']).reset_index(drop=True)

    print(f"  Combined dataset shape: {combined.shape}")

    # 1. Date Features
    print("  • Creating date features...")
    combined['year'] = combined['month'].dt.year
    combined['month_num'] = combined['month'].dt.month
    combined['quarter'] = combined['month'].dt.quarter
    combined['month_sin'] = np.sin(2 * np.pi * combined['month_num'] / 12)
    combined['month_cos'] = np.cos(2 * np.pi * combined['month_num'] / 12)
    combined['days_in_month'] = combined['month'].dt.days_in_month

    # Month index (sequential counter)
    combined['month_index'] = (combined['year'] - combined['year'].min()) * 12 + combined['month_num']

    # 2. Lag Features
    print("  • Creating lag features...")
    lag_periods = [1, 2, 3, 6, 12]

    for lag in lag_periods:
        combined[f'lag_{lag}'] = combined.groupby('sector')['amount_new_house_transaction'].shift(lag)

    # 3. Rolling Statistics
    print("  • Creating rolling statistics...")
    for window in [3, 6, 12]:
        combined[f'rolling_mean_{window}'] = combined.groupby('sector')['amount_new_house_transaction'].transform(
            lambda x: x.shift(1).rolling(window=window, min_periods=1).mean()
        )
        combined[f'rolling_std_{window}'] = combined.groupby('sector')['amount_new_house_transaction'].transform(
            lambda x: x.shift(1).rolling(window=window, min_periods=1).std()
        )

    # 4. Expanding Statistics (cumulative)
    print("  • Creating expanding statistics...")
    combined['expanding_mean'] = combined.groupby('sector')['amount_new_house_transaction'].transform(
        lambda x: x.shift(1).expanding(min_periods=1).mean()
    )
    combined['expanding_std'] = combined.groupby('sector')['amount_new_house_transaction'].transform(
        lambda x: x.shift(1).expanding(min_periods=1).std()
    )

    # 5. Trend Features
    print("  • Creating trend features...")
    combined['diff_1'] = combined.groupby('sector')['amount_new_house_transaction'].diff(1)
    combined['diff_12'] = combined.groupby('sector')['amount_new_house_transaction'].diff(12)
    combined['pct_change_1'] = combined.groupby('sector')['amount_new_house_transaction'].pct_change(1)

    # 6. Merge Pre-owned House Data
    print("  • Merging pre-owned house data...")
    preowned_agg = preowned.groupby(['month', 'sector']).agg({
        'amount_pre_owned_house_transaction': ['mean', 'sum', 'count'],
        'avg_price_per_sqm_pre_owned_house': ['mean', 'std']
    }).reset_index()
    preowned_agg.columns = ['month', 'sector', 'preowned_amount_mean', 'preowned_amount_sum',
                            'preowned_count', 'preowned_price_mean', 'preowned_price_std']

    combined = combined.merge(preowned_agg, on=['month', 'sector'], how='left')

    # 7. Merge Land Transaction Data
    print("  • Merging land transaction data...")
    land_agg = land.groupby(['month', 'sector']).agg({
        'amount_land_transaction': ['mean', 'sum', 'count'],
        'avg_price_per_sqm_land': ['mean', 'std']
    }).reset_index()
    land_agg.columns = ['month', 'sector', 'land_amount_mean', 'land_amount_sum',
                        'land_count', 'land_price_mean', 'land_price_std']

    combined = combined.merge(land_agg, on=['month', 'sector'], how='left')

    # 8. Create Ratio Features
    print("  • Creating ratio features...")
    combined['new_to_preowned_ratio'] = combined['lag_1'] / (combined['preowned_amount_mean'] + 1)
    combined['land_to_new_ratio'] = combined['land_amount_mean'] / (combined['lag_1'] + 1)

    # 9. Merge Sector POI Data
    print("  • Merging sector POI data...")
    sector_poi = data['sector_poi']
    if 'sector' in sector_poi.columns:
        poi_features = sector_poi.copy()
        combined = combined.merge(poi_features, on='sector', how='left')

    # 10. Merge City-level Data
    print("  • Merging city-level data...")
    # Note: Assumes city can be mapped from sector or is in the data
    # This is a placeholder - adjust based on actual data structure

    # 11. Target Encoding for Sector
    print("  • Creating target encoding...")
    sector_mean = combined[combined['is_train'] == 1].groupby('sector')['amount_new_house_transaction'].mean()
    combined['sector_target_mean'] = combined['sector'].map(sector_mean)

    # 12. Sector Statistics
    sector_stats = combined[combined['is_train'] == 1].groupby('sector')['amount_new_house_transaction'].agg([
        'mean', 'std', 'min', 'max', 'count'
    ]).reset_index()
    sector_stats.columns = ['sector', 'sector_mean', 'sector_std', 'sector_min', 'sector_max', 'sector_count']
    combined = combined.merge(sector_stats, on='sector', how='left')

    print(f"\n✓ Feature engineering complete! Final shape: {combined.shape}")
    print(f"  Total features: {combined.shape[1]}")

    return combined

# Engineer features
df_features = engineer_features(data)


In [ ]:

def preprocess_data(df):
    """Preprocess and split data"""

    print("\nPreprocessing data...")

    # Split back to train and test
    train = df[df['is_train'] == 1].copy()
    test = df[df['is_train'] == 0].copy()

    print(f"  Train shape: {train.shape}")
    print(f"  Test shape: {test.shape}")

    # Define feature columns (exclude target, identifiers, and datetime)
    exclude_cols = ['month_sector', 'month', 'amount_new_house_transaction', 'is_train']
    feature_cols = [col for col in train.columns if col not in exclude_cols]

    print(f"\n  Total feature columns: {len(feature_cols)}")

    # Handle missing values
    print("\n  Handling missing values...")

    # For numeric columns, fill with median
    numeric_cols = train[feature_cols].select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        median_val = train[col].median()
        train[col].fillna(median_val, inplace=True)
        test[col].fillna(median_val, inplace=True)

    # For categorical columns, fill with mode
    categorical_cols = train[feature_cols].select_dtypes(include=['object']).columns
    for col in categorical_cols:
        mode_val = train[col].mode()[0] if len(train[col].mode()) > 0 else 'Unknown'
        train[col].fillna(mode_val, inplace=True)
        test[col].fillna(mode_val, inplace=True)

        # Label encode
        le = LabelEncoder()
        train[col] = le.fit_transform(train[col].astype(str))
        test[col] = le.transform(test[col].astype(str))

    print(f"  Missing values in train: {train[feature_cols].isnull().sum().sum()}")
    print(f"  Missing values in test: {test[feature_cols].isnull().sum().sum()}")

    # Create time-based validation split (last 6 months for validation)
    train = train.sort_values('month')
    split_date = train['month'].max() - pd.DateOffset(months=6)

    train_data = train[train['month'] < split_date]
    val_data = train[train['month'] >= split_date]

    X_train = train_data[feature_cols]
    y_train = train_data['amount_new_house_transaction']

    X_val = val_data[feature_cols]
    y_val = val_data['amount_new_house_transaction']

    X_test = test[feature_cols]

    print(f"\n  Time-based split:")
    print(f"    Training: {X_train.shape[0]} samples (before {split_date.strftime('%Y-%m')})")
    print(f"    Validation: {X_val.shape[0]} samples (from {split_date.strftime('%Y-%m')})")
    print(f"    Test: {X_test.shape[0]} samples")

    return X_train, y_train, X_val, y_val, X_test, test, feature_cols

# Preprocess data
X_train, y_train, X_val, y_val, X_test, test_df, features = preprocess_data(df_features)


## 6. MODEL DEVELOPMENT

In [ ]:
def custom_mape(y_true, y_pred):
    """Calculate MAPE (Mean Absolute Percentage Error)"""
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def train_lightgbm(X_train, y_train, X_val, y_val):
    """Train LightGBM model"""
    print("\n1. Training LightGBM...")

    params = {
        'objective': 'regression',
        'metric': 'mape',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1,
        'random_state': RANDOM_SEED
    }

    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

    model = lgb.train(
        params,
        train_data,
        num_boost_round=1000,
        valid_sets=[train_data, val_data],
        valid_names=['train', 'valid'],
        callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(100)]
    )

    return model

def train_xgboost(X_train, y_train, X_val, y_val):
    """Train XGBoost model"""
    print("\n2. Training XGBoost...")

    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'mape',
        'max_depth': 6,
        'learning_rate': 0.05,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'random_state': RANDOM_SEED
    }

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)

    model = xgb.train(
        params,
        dtrain,
        num_boost_round=1000,
        evals=[(dtrain, 'train'), (dval, 'valid')],
        early_stopping_rounds=50,
        verbose_eval=100
    )

    return model

def train_catboost(X_train, y_train, X_val, y_val):
    """Train CatBoost model"""
    print("\n3. Training CatBoost...")

    model = CatBoostRegressor(
        iterations=1000,
        learning_rate=0.05,
        depth=6,
        loss_function='MAPE',
        random_seed=RANDOM_SEED,
        verbose=100,
        early_stopping_rounds=50
    )

    model.fit(
        X_train, y_train,
        eval_set=(X_val, y_val),
        use_best_model=True
    )

    return model

# Train models
models = {}
models['lgb'] = train_lightgbm(X_train, y_train, X_val, y_val)
models['xgb'] = train_xgboost(X_train, y_train, X_val, y_val)
models['catboost'] = train_catboost(X_train, y_train, X_val, y_val)

## 7. MODEL EVALUATION

In [ ]:
def evaluate_models(models, X_val, y_val):
    """Evaluate all models"""

    results = {}
    predictions = {}

    print("\nModel Performance on Validation Set:")
    print("-" * 80)

    for name, model in models.items():
        if name == 'lgb':
            y_pred = model.predict(X_val, num_iteration=model.best_iteration)
        elif name == 'xgb':
            dval = xgb.DMatrix(X_val)
            y_pred = model.predict(dval)
        else:  # catboost
            y_pred = model.predict(X_val)

        # Ensure non-negative predictions
        y_pred = np.maximum(y_pred, 0)

        mape = custom_mape(y_val.values, y_pred)
        r2 = r2_score(y_val.values, y_pred)
        rmse = np.sqrt(mean_squared_error(y_val.values, y_pred))

        results[name] = {'MAPE': mape, 'R2': r2, 'RMSE': rmse}
        predictions[name] = y_pred

        print(f"\n{name.upper()}:")
        print(f"  MAPE: {mape:.4f}%")
        print(f"  R²: {r2:.4f}")
        print(f"  RMSE: {rmse:.2f}")

    # Visualization
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    for idx, (name, y_pred) in enumerate(predictions.items()):
        axes[idx].scatter(y_val, y_pred, alpha=0.5, s=10)
        axes[idx].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()],
                      'r--', lw=2, label='Perfect Prediction')
        axes[idx].set_xlabel('Actual', fontsize=12)
        axes[idx].set_ylabel('Predicted', fontsize=12)
        axes[idx].set_title(f'{name.upper()} - R²: {results[name]["R2"]:.4f}',
                           fontsize=14, fontweight='bold')
        axes[idx].legend()
        axes[idx].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('model_predictions.png', dpi=300, bbox_inches='tight')
    print("\n✓ Prediction plots saved as 'model_predictions.png'")

    return results, predictions

# Evaluate models
eval_results, val_predictions = evaluate_models(models, X_val, y_val)

# Feature Importance (LightGBM)
print("\n" + "-" * 80)
print("TOP 20 MOST IMPORTANT FEATURES (LightGBM)")
print("-" * 80)

importance_df = pd.DataFrame({
    'feature': features,
    'importance': models['lgb'].feature_importance(importance_type='gain')
}).sort_values('importance', ascending=False).head(20)

print(importance_df.to_string(index=False))

# Plot feature importance
plt.figure(figsize=(10, 8))
plt.barh(importance_df['feature'], importance_df['importance'])
plt.xlabel('Importance (Gain)', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('Top 20 Feature Importance (LightGBM)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
print("\n✓ Feature importance plot saved as 'feature_importance.png'")

In [1]:
kaggle competitions submit -c china-real-estate-demand-prediction -f submission.csv -m "submit"

SyntaxError: invalid syntax (ipython-input-46244318.py, line 1)